# Question 3

Oilco has oil fields in Los Angeles and San Diego. The Los Angeles field can produce 400,000 barrels
per day and the San Diego field can produce 500,000 barrels per day.

Oil is sent from the fields to a refinery, in either Dallas or Houston (assume each refinery has unlimited
capacity). To refine 100,000 barrels costs $700 at Dallas and $900 at Houston.

Refined oil is shipped to customers in New York and Chicago. New York customers require 300,000
barrels per day and Chicago customers require 400,000 barrels per day.

The costs of shipping 100,000 barrels of oil (refined or unrefined) between cities are shown below.

|From/To($) | Dallas | Houston | New York | Chicago |
|:---------:|:------:|:-------:|:--------:|:-------:|
|Los Angels |300     |   110   |    -     |     -   |
|San Diego  |420     |   100   |    -     |    -    |
|Dallas     |-       |   -     |   450    |   550   |
|Houston    |-       |   -     |   470    | 530     |

In [1]:
using JuMP, HiGHS

# defining model
oilco = Model(HiGHS.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

In [2]:
numfields = 2
numreceivers = 4
numedges = numfields*numreceivers

#Define the incidence matrix A for our fully connected network
#Edges go from the fields (first 2 rows) to the refineries (next 2 rows) to the customers (last 2 rows)
#Edges go from the power plants (first 3 rows) to the cities (last 4 rows)
#Edge 1 is column 1, edge 2 is column 2, etc.
A = [1  1  0  0  0  0  0  0
     0  0  1  1  0  0  0  0
    -1  0 -1  0  1  1  0  0 
     0 -1  0 -1  0  0  1  1 
     0  0  0  0 -1  0 -1  0
     0  0  0  0  0 -1  0 -1]

#Supply (first 2), relay (middle 2), demand (last 2)
b = [400000, 500000, 0, 0, -300000, -400000]

# costs of shipping per 100000 barrels
costs = [300 110 420 100 450 550 470 530];

A

6×8 Matrix{Int64}:
  1   1   0   0   0   0   0   0
  0   0   1   1   0   0   0   0
 -1   0  -1   0   1   1   0   0
  0  -1   0  -1   0   0   1   1
  0   0   0   0  -1   0  -1   0
  0   0   0   0   0  -1   0  -1

In [3]:
@variable(oilco, x[1:numedges] >= 0)

@constraint(oilco, supplyanddemand, A*x .== b)

@objective(oilco, Min, sum(costs[i]*x[i] for i in 1:numedges))

print(oilco)

In [4]:
optimize!(oilco)

Presolving model
2 rows, 4 cols, 8 nonzeros
Problem status detected on presolve: Infeasible
Model   status      : Infeasible
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
ERROR:   No invertible representation for getDualRay
